## 1) Instalar dependências

In [ ]:
!python -m pip install --upgrade pip
!pip install -r requirements-colab.txt

### (Opcional) Instalar PyTorch com suporte CUDA

Se usar GPU no Colab, comente a célula anterior e execute abaixo (ajuste cu118 conforme versão CUDA do runtime).

In [ ]:
# !pip install --index-url https://download.pytorch.org/whl/cu118 torch torchvision torchaudio --upgrade
print('Instale PyTorch com CUDA se tiver GPU disponível.')

## 2) Montar Google Drive e criar pastas

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# Criar pasta para salvar runs
run_dir = '/content/drive/MyDrive/agent_runs'
os.makedirs(run_dir, exist_ok=True)
print(f'Runs serão salvos em: {run_dir}')

## 3) Clonar repositório e definir diretório de trabalho

In [ ]:
# Se o repo não estiver presente, clonar do GitHub
!git clone https://github.com/devolopbomfim/Xadrez_AI_Final.git || echo 'Repo já existe ou clone falhou'

## 4) Definir WORKDIR e mudar para ele

In [ ]:
import os

WORKDIR = '/content/Xadrez_AI_Final'

if os.path.exists(WORKDIR):
    %cd /content/Xadrez_AI_Final
    print(f'Working directory: {os.getcwd()}')
else:
    print(f'Erro: Repo não encontrado em {WORKDIR}')

## 5) Checar GPU e ambiente

In [ ]:
!nvidia-smi || echo 'GPU não detectada'
import torch
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

## 6) Criar estrutura de diretórios para AgentA

In [ ]:
!mkdir -p models/AgentA
!mkdir -p models/AgentA/checkpoints
!mkdir -p models/AgentA/experiments
print('Diretórios criados para AgentA')

## 7) Configuração de hiperparâmetros (arquivo YAML)

In [ ]:
cfg = '''learning_rate: 0.001
batch_size: 128
gamma: 0.99
total_timesteps: 100000
seed: 42
checkpoint_interval: 500
device: cuda
iters_per_epoch: 100
'''

with open('colab_train_config.yaml', 'w') as f:
    f.write(cfg)

print('Arquivo colab_train_config.yaml criado')

## 8) Smoke test rápido — Forward pass do modelo

In [ ]:
import torch
import numpy as np
from training.model import make_model

m = make_model(device='cpu')
x = np.random.randn(1, 13, 8, 8).astype('float32')
t = torch.tensor(x)
pi, v = m(t)

assert pi is not None and v is not None, 'Forward pass falhou'
print(f'Smoke test PASSOU')
print(f'  Policy logits shape: {pi.shape}')
print(f'  Value shape: {v.shape}')

## 9) Exemplo: Rodar self-play + treino (run_iteration)

**IMPORTANTE:** Use `python3 -m training.run_iteration` (não `python3 training/run_iteration.py`)

Ajuste os parâmetros conforme necessário:
- `--num-selfplay`: número de jogos de self-play (padrão: 5)
- `--selfplay-sims`: simulações por move no MCTS (padrão: 50)
- `--trainer-iters`: iterações de treino (padrão: 100)
- `--batch-size`: tamanho do batch (padrão: 128)

In [ ]:
AGENT_DIR = 'models/AgentA'

# Executar via módulo (correto para Colab)
!python3 -m training.run_iteration {AGENT_DIR} --iteration 0 --num-selfplay 2 --selfplay-sims 30 --trainer-iters 100 --batch-size 128

## 10) Salvar checkpoints no Google Drive

In [ ]:
import shutil
import glob
import os

ckpt_src = 'models/AgentA/checkpoints'
ckpt_dst = '/content/drive/MyDrive/agent_runs/AgentA_checkpoints'

os.makedirs(ckpt_dst, exist_ok=True)

# Copiar todos os .pt files
for f in glob.glob(os.path.join(ckpt_src, '*.pt')):
    shutil.copy(f, ckpt_dst)
    print(f'Copiado: {os.path.basename(f)}')

print(f'\nCheckpoints salvos em: {ckpt_dst}')

## 11) TensorBoard — Visualizar logs (opcional)

In [ ]:
# Descomente para usar TensorBoard
# %load_ext tensorboard
# %tensorboard --logdir models/AgentA/logs --host 0.0.0.0 --port 6006
print('TensorBoard está pronto. Descomente as linhas acima para visualizar logs.')

## 12) Avaliação — Rodar episódios de teste

In [ ]:
from core.board.board import Board

# Exemplo simples: criar e avaliar um board
board = Board()
print(f'Board inicial criado')
print(f'Lado a mover: {"Branco" if board.side_to_move == 0 else "Preto"}')
print('Use training/run_iteration.py para avaliação completa com arena matches')